### Convert embeddings into raw data
Use the following to convert word embedding using's FaceBooks's fastText into raw data for a standard machine learning model. We are assuming a closed world solution with only positive and negative values for our 5 predicates - isChild, isSpouse, isFather, isMother, and isSibling.

In [2]:
import sklearn as sk 
import scipy as sci 
import numpy as np 
import pandas as pd
import pickle

Load in the pickle files containing the word encodings and the entity map

In [77]:
words = pickle.load(open('embeddings/word_vectors.pkl','rb'))
key = pickle.load(open('processed/large_set/entities_map_large.pkl','rb'))

Load in the files containing the positive and negative triplets of (entity,predicate,entity)

In [4]:
converter = dict.fromkeys(range(4), lambda s: s.decode('utf-8'))
pos = np.loadtxt(r'processed/large_set/positiveTriplets_unique_large.txt',delimiter=' ',dtype=str, converters=converter)
neg = np.loadtxt(r'processed/large_set/negativeTriplets_more_large.txt',delimiter=' ',dtype=str, converters=converter)
pos.shape,neg.shape

((37395, 4), (37391, 4))

Make a list of the entities for looping over all of them later

In [5]:
pred_key = {'P22':'isFather','P25':'isMother','P26':'isSpouse','P3373':'isSibling','P40':'isChild'}
pred_key

{'P22': 'isFather',
 'P25': 'isMother',
 'P26': 'isSpouse',
 'P3373': 'isSibling',
 'P40': 'isChild'}

In [31]:
entities = np.concatenate([np.unique(pos[:,0]),np.unique(neg[:,0]),np.unique(pos[:,2]),np.unique(neg[:,2])])
predicates = np.unique(pos[:,1])
entities.shape,predicates.shape

((32363,), (5,))

A few helper functions for the main data converting tool

In [81]:
def encode_word(word):
    """
        Encodes a word if found in the word 
        encoding dictionary; otherwise returns
        an empty array.
    """
    if word in words.keys():
        return(words[word])
    max_len = max([words[x].shape[0] for x in words])
    return(np.empty(max_len))

def encode_entity(entity):
    """
        Builds an entity encoding by computing
        the average of all the word encodings.
        Ignores any words that don't have an 
        encoding in the words_vectors.txt file.
    """
    all_words = np.array([encode_word(x) for x in key[entity].split(' ') if x in words.keys()])
    if all_words.shape[0]==0:
        return(np.array([np.NaN]*300,dtype=float).transpose())
    return(all_words.mean(axis=0))

### Primary Function
The build_rows function builds the appropriate rows of numerical data for a given entity. This is done by considering all triplets with this entity as the first object. Then it encodes all entities in this subset, including itself, and concatenates each encoding. Finally it looks as the predicate for each entity-entity pair and assigns 1,0, or -1 depending on if the predicate is true, unknown, or false for that entity-entity pair. The result is a small dataframe of all the rows of this form where the first entity is the argument.

In [67]:
def build_rows(entity):
    """
        Builds a dataframe out of the triplets
        containing entity as the first object.
    """
    encoding = encode_entity(entity)
    temp_pos = pos[pos[:,0]==entity,:]
    temp_neg = neg[neg[:,0]==entity,:]
    pairs = []
    rows = []
    bad_encodings = []
    for each_obj in np.concatenate([np.unique(temp_pos[:,2]),np.unique(temp_neg[:,2])]):        
        try:
            obj_encode = encode_entity(each_obj)
            
            if not all(np.isnan(obj_encode)):
                # Get the positive & negative triplet's objects
                temp2_pos = temp_pos[temp_pos[:,2]==each_obj,:]
                temp2_neg = temp_neg[temp_neg[:,2]==each_obj,:]

                # Build outputs from which predicates are in triplets
                pos_preds = np.array([x in np.unique(temp2_pos[:,1]) for x in predicates],dtype=int)
                neg_preds = np.array([x in np.unique(temp2_neg[:,1]) for x in predicates],dtype=int)
                preds = pos_preds - neg_preds

                # Update the data
                row = np.concatenate([encoding,obj_encode,preds])
                rows.append(row)
                pairs.append([entity,each_obj])
                
            else:
                #print('Entity ' + obj_encode + ' has no encoding!')
                bad_encodings.append(each_obj)
        except:
            #print("didn't work for "+ str(entity) + ", " + str(each_obj))  
            bad_encodings.append(each_obj)
 
    rows = np.array(rows)
    pairs = np.array(pairs)
    if rows.shape[0] == 0:
        return({'df':np.NaN,'bad_encodings':np.NaN})
    df = pd.concat([pd.DataFrame(pairs),pd.DataFrame(rows)],ignore_index=True,axis=1)
    col_names = ['EntityA','EntityB']+['a'+str(x) for x in range(300)]+['b'+str(x) for x in range(300)]+['p'+str(x) for x in range(5)]
    df.columns = col_names
    return({'df':df,'bad_encodings':bad_encodings})

entity = "Q107507"
test = build_rows(entity)['df']
test.shape

(27, 607)

Running build_rows on all the entities to generate the main dataset

In [ ]:
all_rows = pd.DataFrame([])
bad_encodings = []
for entity in entities:
    rows = build_rows(entity)
    try:
        all_rows = pd.concat([all_rows,rows['df']])
        [bad_encodings.append(x) for x in rows['bad_encodings']]
    except:
        print ('Entity: '+entity+ ' failed!')
        bad_encodings.append(entity)
        
bad_encodings = np.unique(np.array(bad_encodings))

Entity: Q1000924 failed!
Entity: Q1001933 failed!
Entity: Q1002528 failed!
Entity: Q100474 failed!
Entity: Q1004751 failed!
Entity: Q100637 failed!
Entity: Q100723 failed!
Entity: Q100737 failed!
Entity: Q100738 failed!
Entity: Q1008160 failed!
Entity: Q100832 failed!
Entity: Q100907 failed!
Entity: Q101014 failed!
Entity: Q1010351 failed!
Entity: Q1010767 failed!
Entity: Q101172 failed!
Entity: Q101354 failed!
Entity: Q1014890 failed!
Entity: Q101581 failed!
Entity: Q1015893 failed!
Entity: Q1016268 failed!
Entity: Q101638 failed!
Entity: Q101646 failed!
Entity: Q101731 failed!
Entity: Q101764 failed!
Entity: Q101772 failed!
Entity: Q101894 failed!
Entity: Q1019229 failed!
Entity: Q102005 failed!
Entity: Q102028 failed!
Entity: Q1021243 failed!
Entity: Q102166 failed!
Entity: Q102176 failed!
Entity: Q10218 failed!
Entity: Q1021962 failed!
Entity: Q102509 failed!
Entity: Q102516 failed!
Entity: Q102518 failed!
Entity: Q102529 failed!
Entity: Q10261689 failed!
Entity: Q10261692 failed!


Entity: Q10907436 failed!
Entity: Q10907833 failed!
Entity: Q10909275 failed!
Entity: Q10910298 failed!
Entity: Q1091124 failed!
Entity: Q1091197 failed!
Entity: Q1091368 failed!
Entity: Q10916600 failed!
Entity: Q10916603 failed!
Entity: Q10916615 failed!
Entity: Q10916619 failed!
Entity: Q10916631 failed!
Entity: Q10916632 failed!
Entity: Q10916640 failed!
Entity: Q10916652 failed!
Entity: Q10916657 failed!
Entity: Q10916664 failed!
Entity: Q10916666 failed!
Entity: Q10916668 failed!
Entity: Q10916674 failed!
Entity: Q10916698 failed!
Entity: Q10918517 failed!
Entity: Q10921761 failed!
Entity: Q109223 failed!
Entity: Q109240 failed!
Entity: Q109241 failed!
Entity: Q10924171 failed!
Entity: Q10931681 failed!
Entity: Q109338 failed!
Entity: Q10934229 failed!
Entity: Q10942053 failed!
Entity: Q10943157 failed!
Entity: Q109468 failed!
Entity: Q10946845 failed!
Entity: Q10946850 failed!
Entity: Q10947171 failed!
Entity: Q1094730 failed!
Entity: Q10947311 failed!
Entity: Q10948512 failed!


Entity: Q1167368 failed!
Entity: Q116740 failed!
Entity: Q1168080 failed!
Entity: Q11702715 failed!
Entity: Q11707742 failed!
Entity: Q11716005 failed!
Entity: Q11716006 failed!
Entity: Q11716007 failed!
Entity: Q11716009 failed!
Entity: Q11716867 failed!
Entity: Q11719581 failed!
Entity: Q11727405 failed!
Entity: Q117329 failed!
Entity: Q1173598 failed!
Entity: Q11737114 failed!
Entity: Q11754917 failed!
Entity: Q1176034 failed!
Entity: Q11763404 failed!
Entity: Q11767039 failed!
Entity: Q1176739 failed!
Entity: Q11768687 failed!
Entity: Q11768697 failed!
Entity: Q11768977 failed!
Entity: Q11769174 failed!
Entity: Q11771684 failed!
Entity: Q11772220 failed!
Entity: Q11772284 failed!
Entity: Q1177456 failed!
Entity: Q1179126 failed!
Entity: Q11791606 failed!
Entity: Q11806 failed!
Entity: Q118087 failed!
Entity: Q11812 failed!
Entity: Q118253 failed!
Entity: Q118299 failed!
Entity: Q118348 failed!
Entity: Q11835630 failed!
Entity: Q11836906 failed!
Entity: Q11837967 failed!
Entity: Q11

Entity: Q1304124 failed!
Entity: Q1304368 failed!
Entity: Q1305838 failed!
Entity: Q130752 failed!
Entity: Q130812 failed!
Entity: Q13090691 failed!
Entity: Q13090954 failed!
Entity: Q1309296 failed!
Entity: Q1310403 failed!
Entity: Q131045 failed!
Entity: Q13104842 failed!
Entity: Q131195 failed!
Entity: Q1312209 failed!
Entity: Q13132371 failed!
Entity: Q131324 failed!
Entity: Q13133 failed!
Entity: Q131355 failed!
Entity: Q1313570 failed!
Entity: Q1313954 failed!
Entity: Q1314455 failed!
Entity: Q13147738 failed!
Entity: Q1315769 failed!
Entity: Q1316528 failed!
Entity: Q1316745 failed!
Entity: Q1317140 failed!
Entity: Q1317501 failed!
Entity: Q1318372 failed!
Entity: Q1320655 failed!
Entity: Q132090 failed!
Entity: Q13216657 failed!
Entity: Q13223865 failed!
Entity: Q132440 failed!
Entity: Q1324672 failed!
Entity: Q1325288 failed!
Entity: Q132548 failed!
Entity: Q1325718 failed!
Entity: Q1325720 failed!
Entity: Q1325736 failed!
Entity: Q1325750 failed!
Entity: Q13264213 failed!
Ent

Entity: Q1503520 failed!
Entity: Q1503795 failed!
Entity: Q15040852 failed!
Entity: Q15040856 failed!
Entity: Q15043373 failed!
Entity: Q1505262 failed!
Entity: Q1505501 failed!
Entity: Q15055459 failed!
Entity: Q150586 failed!
Entity: Q150620 failed!
Entity: Q150642 failed!
Entity: Q1506551 failed!
Entity: Q1506598 failed!
Entity: Q1506606 failed!
Entity: Q1506637 failed!
Entity: Q15070044 failed!
Entity: Q15070048 failed!
Entity: Q15072599 failed!
Entity: Q15073265 failed!
Entity: Q1507346 failed!
Entity: Q15079141 failed!
Entity: Q1507982 failed!
Entity: Q15088507 failed!
Entity: Q15091752 failed!
Entity: Q1509334 failed!
Entity: Q15094075 failed!
Entity: Q150989 failed!
Entity: Q15103558 failed!
Entity: Q1510784 failed!
Entity: Q151090 failed!
Entity: Q1511 failed!
Entity: Q1511219 failed!
Entity: Q151130 failed!
Entity: Q151171 failed!
Entity: Q151173 failed!
Entity: Q151305 failed!
Entity: Q15139424 failed!
Entity: Q151457 failed!
Entity: Q15149340 failed!
Entity: Q151509 failed!

...

Entity: Q4689378 failed!
Entity: Q469013 failed!
Entity: Q469041 failed!
Entity: Q469059 failed!
Entity: Q469131 failed!
Entity: Q4692991 failed!
Entity: Q4693031 failed!
Entity: Q4693143 failed!
Entity: Q469354 failed!
Entity: Q469359 failed!
Entity: Q469389 failed!
Entity: Q469397 failed!
Entity: Q469421 failed!
Entity: Q469437 failed!
Entity: Q469607 failed!
Entity: Q469611 failed!
Entity: Q469622 failed!
Entity: Q469631 failed!
Entity: Q469638 failed!
Entity: Q469643 failed!
Entity: Q469648 failed!
Entity: Q4696632 failed!
Entity: Q469701 failed!
Entity: Q469702 failed!
Entity: Q469742 failed!
Entity: Q469760 failed!
Entity: Q469783 failed!
Entity: Q469830 failed!
Entity: Q469896 failed!
Entity: Q469955 failed!
Entity: Q4699958 failed!
Entity: Q470031 failed!
Entity: Q470034 failed!
Entity: Q470198 failed!
Entity: Q4702270 failed!
Entity: Q470261 failed!
Entity: Q470272 failed!
Entity: Q470283 failed!
Entity: Q470293 failed!
Entity: Q470362 failed!
Entity: Q4704127 failed!
Entity: 

Entity: Q5041442 failed!
Entity: Q504276 failed!
Entity: Q5045147 failed!
Entity: Q5045269 failed!
Entity: Q504535 failed!
Entity: Q504556 failed!
Entity: Q504733 failed!
Entity: Q504792 failed!
Entity: Q505005 failed!
Entity: Q505101 failed!
Entity: Q505112 failed!
Entity: Q505178 failed!
Entity: Q5052553 failed!
Entity: Q5052733 failed!
Entity: Q5052999 failed!
Entity: Q5053006 failed!
Entity: Q5056508 failed!
Entity: Q5056518 failed!
Entity: Q506060 failed!
Entity: Q506339 failed!
Entity: Q506354 failed!
Entity: Q506415 failed!
Entity: Q5064594 failed!
Entity: Q506466 failed!
Entity: Q506514 failed!
Entity: Q5065971 failed!
Entity: Q506720 failed!
Entity: Q5071309 failed!
Entity: Q5072084 failed!
Entity: Q507243 failed!
Entity: Q507315 failed!
Entity: Q5074968 failed!
Entity: Q5075253 failed!
Entity: Q507801 failed!
Entity: Q5080420 failed!
Entity: Q5080867 failed!
Entity: Q5081413 failed!
Entity: Q5081465 failed!
Entity: Q508212 failed!
Entity: Q5082733 failed!
Entity: Q5082734 fai

Entity: Q542520 failed!
Entity: Q542672 failed!
Entity: Q542695 failed!
Entity: Q542719 failed!
Entity: Q542742 failed!
Entity: Q542744 failed!
Entity: Q542773 failed!
Entity: Q5428603 failed!
Entity: Q5428633 failed!
Entity: Q5429464 failed!
Entity: Q5429531 failed!
Entity: Q5429658 failed!
Entity: Q5431091 failed!
Entity: Q5431346 failed!
Entity: Q543564 failed!
Entity: Q543648 failed!
Entity: Q543699 failed!
Entity: Q5437870 failed!
Entity: Q5437879 failed!
Entity: Q5438354 failed!
Entity: Q543880 failed!
Entity: Q543922 failed!
Entity: Q544055 failed!
Entity: Q544075 failed!
Entity: Q544086 failed!
Entity: Q544105 failed!
Entity: Q544152 failed!
Entity: Q544166 failed!
Entity: Q544228 failed!
Entity: Q5444165 failed!
Entity: Q544496 failed!
Entity: Q544609 failed!
Entity: Q5447176 failed!
Entity: Q544951 failed!
Entity: Q545140 failed!
Entity: Q545755 failed!
Entity: Q546008 failed!
Entity: Q5460298 failed!
Entity: Q5460608 failed!
Entity: Q546082 failed!
Entity: Q5464191 failed!
E

Entity: Q576388 failed!
Entity: Q576479 failed!
Entity: Q57651 failed!
Entity: Q57654 failed!
Entity: Q57662 failed!
Entity: Q576665 failed!
Entity: Q576692 failed!
Entity: Q57671 failed!
Entity: Q5768127 failed!
Entity: Q5768604 failed!
Entity: Q57713 failed!
Entity: Q577162 failed!
Entity: Q5771862 failed!
Entity: Q577795 failed!
Entity: Q5778165 failed!
Entity: Q57791 failed!
Entity: Q577912 failed!
Entity: Q5779373 failed!
Entity: Q57795 failed!
Entity: Q578007 failed!
Entity: Q57841 failed!
Entity: Q57845 failed!
Entity: Q5784538 failed!
Entity: Q5784543 failed!
Entity: Q5787555 failed!
Entity: Q578791 failed!
Entity: Q57909 failed!
Entity: Q57913 failed!
Entity: Q57943 failed!
Entity: Q579490 failed!
Entity: Q57950 failed!
Entity: Q579596 failed!
Entity: Q579846 failed!
Entity: Q58001 failed!
Entity: Q580014 failed!
Entity: Q580075 failed!
Entity: Q58010 failed!
Entity: Q5801473 failed!
Entity: Q580249 failed!
Entity: Q5804050 failed!
Entity: Q5805275 failed!
Entity: Q58056 faile

Entity: Q62153 failed!
Entity: Q621548 failed!
Entity: Q62168 failed!
Entity: Q6219733 failed!
Entity: Q6222299 failed!
Entity: Q6224145 failed!
Entity: Q6225119 failed!
Entity: Q62259 failed!
Entity: Q62261 failed!
Entity: Q6227170 failed!
Entity: Q62274 failed!
Entity: Q623135 failed!
Entity: Q6231704 failed!
Entity: Q623233 failed!
Entity: Q6233263 failed!
Entity: Q62393 failed!
Entity: Q6245623 failed!
Entity: Q6245625 failed!
Entity: Q6246599 failed!
Entity: Q62477 failed!
Entity: Q62480 failed!
Entity: Q62481 failed!
Entity: Q62483 failed!
Entity: Q62499 failed!
Entity: Q62511 failed!
Entity: Q62523 failed!
Entity: Q62529 failed!
Entity: Q62530 failed!
Entity: Q62533 failed!
Entity: Q62549 failed!
Entity: Q62550 failed!
Entity: Q62566 failed!
Entity: Q625661 failed!
Entity: Q62568 failed!
Entity: Q625756 failed!
Entity: Q62579 failed!
Entity: Q6258771 failed!
Entity: Q6263425 failed!
Entity: Q626461 failed!
Entity: Q62649 failed!
Entity: Q62650 failed!
Entity: Q62659 failed!
Enti

Save the data to a file

In [149]:
all_rows.to_csv('EncodedDataLrg.csv',index=False)

Encode all entities and save as pickle file for future use

In [82]:
Encoded = {entity:encode_entity(entity) for entity in entities}
pickle.dump( Encoded, open( "embeddings/entity_embeddings_lrg.pkl", "wb" ) )
pickle.dump( bad_encodings, open( "embeddings/bad_entities_lrg.pkl", "wb") )